## Topic Modelling Gensim

In [7]:
import gensim
import gensim.corpora as corpora
import pickle
import pandas as pd
from pprint import pprint
from gensim.models import CoherenceModel
import csv

In [ ]:
# select tokens and subsample
tokenlist = pd.read_pickle("dataset_with_tokens.pkl")[["page", "tokens"]]
tokenlist = tokenlist["tokens"].tolist()

In [ ]:
# make bigram and trigram mod

# is that necessary?
bigram = gensim.models.Phrases(tokenlist, min_count = 5, threshold = 100)
trigram = gensim.models.Phrases(bigram[tokenlist], threshold = 100)

trigram_mod = gensim.models.phrases.Phraser(trigram)  
bigram_mod = gensim.models.phrases.Phraser(bigram)

tokenlist = [trigram_mod[bigram_mod[text]] for text in tokenlist]

In [ ]:
tokens_trigrams = []
for doc in tokenlist:
    tokenstring = [" ".join([token for token in doc])]
    tokens_trigrams.append(tokenstring)

In [ ]:
with open("tokens/tokens_trigrams.csv", "w") as f:
    wr = csv.writer(f)
    wr.writerows(tokens_trigrams)

In [ ]:
# create dictionary (each word recorded once and connected to id)
id2word = corpora.Dictionary(tokenlist)
id2word.save("gensim_models/id2word")

# create corpus
corpus = [id2word.doc2bow(text) for text in tokenlist]
import pickle
with open("gensim_models/corpus", "wb") as f:
    pickle.dump(corpus, f)


In [2]:
# load dictionary
id2word = gensim.corpora.dictionary.Dictionary.load("gensim_models/id2word")

# load corpus
class MyCorpus:
    def __iter__(self):
        for line in open("tokens/tokens_trigrams.csv"):
            # assume there's one document per line, tokens separated by whitespace
            yield id2word.doc2bow(line.lower().split())

corpus = MyCorpus()


In [3]:
# build LDA model

lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=22,
                                           random_state=100,
                                           eval_every = 1,
                                           chunksize = 100,
                                           passes = 10,
                                           alpha = "symmetric",
                                           per_word_topics = True,
                                           workers = 2)

In [4]:
lda_model.save("gensim_models/lda_22_topics")

In [5]:
modell =  gensim.models.ldamodel.LdaModel.load("gensim_models/lda_22_topics")

In [9]:
from gensim.test.utils import datapath

# Save model to disk.
temp_file = datapath("gensim_models/lda_22")
lda_model.save(temp_file)

FileNotFoundError: [Errno 2] No such file or directory: '/home/hennes/.local/lib/python3.8/site-packages/gensim/test/test_data/gensim_models/lda_22.state'

In [4]:
# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [11]:
?modell.print_topics

In [8]:
# print word loadings on each topic
pprint(modell.print_topics())

[(12,
  '0.051*"nicaragua" + 0.050*"país" + 0.024*"internacional" + '
  '0.019*"económico" + 0.015*"gobierno" + 0.013*"región" + 0.012*"mundial" + '
  '0.012*"unidos" + 0.011*"méxico" + 0.011*"mundo"'),
 (2,
  '0.027*"nicaragua" + 0.018*"político" + 0.018*"régimen" + 0.016*"gobierno" + '
  '0.013*"periodista" + 0.012*"unidos" + 0.011*"preso" + 0.011*"derecho" + '
  '0.011*"país" + 0.010*"nicaragüense"'),
 (15,
  '0.073*"ortega" + 0.069*"nicaragua" + 0.048*"daniel" + 0.034*"gobierno" + '
  '0.028*"presidente" + 0.024*"pueblo" + 0.020*"nacional" + 0.017*"sandinista" '
  '+ 0.016*"nicaragüense" + 0.014*"rosario_murillo"'),
 (13,
  '0.029*"mujer" + 0.027*"joven" + 0.024*"hombre" + 0.022*"encontrar" + '
  '0.019*"salir" + 0.015*"vida" + 0.013*"casa" + 0.013*"persona" + '
  '0.012*"llegar" + 0.012*"llevar"'),
 (9,
  '0.067*"vivienda" + 0.064*"managua" + 0.051*"barrio" + 0.047*"casa" + '
  '0.038*"familia" + 0.036*"calle" + 0.031*"alcaldía" + 0.019*"azul_blanco" + '
  '0.017*"comerciante" + 0

In [ ]:
# not sure what this saves
doc_lda = lda_model[corpus]

### Model Performance

In [15]:
corrpus = (row for row in open("tokens/tokens_trigrams.csv"))

In [18]:
# Compute Perplexity
print('\nPerplexity: ', modell.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

TypeError: object of type 'MyCorpus' has no len()

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenlist, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### Visualising Model

In [12]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

with open("gensim_models/corpus", "rb") as f:
    corpus = pickle.load(f)

In [13]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(modell, corpus, id2word)
vis

/home/hennes/.local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.199742  0.079102       1        1  12.325091
15    -0.162877 -0.068471       2        1   8.581935
6     -0.148134  0.173229       3        1   8.182809
14    -0.084898 -0.102553       4        1   5.917887
12    -0.182762 -0.128972       5        1   5.679074
10    -0.036562 -0.065760       6        1   4.923946
18     0.058398  0.068737       7        1   4.703869
16     0.056284  0.054291       8        1   4.608687
5     -0.011541 -0.157716       9        1   4.538256
4     -0.109942  0.094492      10        1   4.459055
3     -0.020373 -0.075288      11        1   4.286996
13     0.007947  0.297042      12        1   4.032157
11    -0.094582 -0.124842      13        1   3.995642
21    -0.008084 -0.004071      14        1   3.807626
17     0.030982  0.111842      15        1   3.704149
1     -0.026534 -0.035721      16        1   3.512235
19    -0.005521  0.075637      17        1   3.451329
20     0.270900 -0.052558      18        1   2.666540
9      0.142041  0.041900      19        1   2.137918
0      0.244856  0.074763      20        1   1.785257
7      0.044133 -0.143724      21        1   1.493802
8      0.236013 -0.111359      22        1   1.205738, topic_info=           Term           Freq          Total Category  logprob  loglift
285   nicaragua  255929.000000  255929.000000  Default  30.0000  30.0000
523      ortega  116441.000000  116441.000000  Default  29.0000  29.0000
259       salud   72602.000000   72602.000000  Default  28.0000  28.0000
103     policía   62370.000000   62370.000000  Default  27.0000  27.0000
1           año  145261.000000  145261.000000  Default  26.0000  26.0000
...         ...            ...            ...      ...      ...      ...
219     atender    2676.793108   17918.116145  Topic22  -4.3161   2.5169
4709    segovia    1890.550715    5730.948103  Topic22  -4.6639   3.3091
187     brindar    2432.275921   23046.060407  Topic22  -4.4119   2.1694
2822       agua    1820.812295   12482.971809  Topic22  -4.7015   2.4930
2723  trasladar    1818.376952   12408.752057  Topic22  -4.7028   2.4976

[1339 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
4178     19  0.998237         abastecer
6451     21  0.999202           abogada
5816     12  0.999676           abogado
5817      5  0.998171  abogado_defensor
1355      1  0.558818             abril
...     ...       ...               ...
1116     18  0.189237              zona
1116     19  0.059101              zona
1116     20  0.104374              zona
4710      4  0.998910     álvaro_vargas
4944      9  0.999588             árbol

[2900 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 16, 7, 15, 13, 11, 19, 17, 6, 5, 4, 14, 12, 22, 18, 2, 20, 21, 10, 1, 8, 9])

## Documents and Topics

In [ ]:
# find dominant model in each doc

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=tokenlist):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=tokenlist)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
# find most representative document for each topic

# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
# topic distribution across documents
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics